<a href="https://colab.research.google.com/github/mmcuervo/ExamMLcategorias/blob/Experimento3/02_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# En este archivo analizamos los datos del dataset

In [2]:
# 1) Importar librerías básicas para EDA
import pandas as pd              # Manejo de datos en DataFrames
import numpy as np               # Utilidades numéricas (vectores, matrices, NaN)
import matplotlib.pyplot as plt  # Gráficas base
import seaborn as sns            # Gráficas estadísticas (opcional, solo EDA)
from pathlib import Path         # Manejo seguro de rutas de archivos


In [3]:
#2 Llamamos al dataset y creamos una copia para trabajar en ella y no tocar el original
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/MyDrive/AiLabData/Support_tickets.csv"
df = pd.read_csv(DATA_PATH, encoding="utf-8")
dc = df.copy()

Mounted at /content/drive


In [4]:
#3 Verificamos el archivo (info, head)
print("Dimensiones del dataset:" )
dc.info()
pd.set_option('display.max_columns', None)
dc.head()

Dimensiones del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ticket_id               50000 non-null  int64  
 1   day_of_week             50000 non-null  object 
 2   day_of_week_num         50000 non-null  int64  
 3   company_id              50000 non-null  int64  
 4   company_size            50000 non-null  object 
 5   company_size_cat        50000 non-null  int64  
 6   industry                50000 non-null  object 
 7   industry_cat            50000 non-null  int64  
 8   customer_tier           50000 non-null  object 
 9   customer_tier_cat       50000 non-null  int64  
 10  org_users               50000 non-null  int64  
 11  region                  50000 non-null  object 
 12  region_cat              50000 non-null  int64  
 13  past_30d_tickets        50000 non-null  int64  
 14  past_90d_inci

,ticket_id,day_of_week,day_of_week_num,company_id,company_size,company_size_cat,industry,industry_cat,customer_tier,customer_tier_cat,org_users,region,region_cat,past_30d_tickets,past_90d_incidents,product_area,product_area_cat,booking_channel,booking_channel_cat,reported_by_role,reported_by_role_cat,customers_affected,error_rate_pct,downtime_min,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,customer_sentiment,customer_sentiment_cat,description_length,priority,priority_cat
0,1000000000,Wed,3,100015,Small,1,media,7,Basic,1,126,APAC,3,2,0,mobile,3,web,1,support,1,2,5.451201,6,0,0,0,0,neutral,2,227,low,1
1,1000000001,Sat,6,100023,Small,1,healthcare,5,Basic,1,101,AMER,1,2,3,analytics,5,chat,3,product_manager,3,24,2.495538,2,0,0,0,0,neutral,2,461,low,1
2,1000000002,Mon,1,100012,Small,1,gaming,4,Basic,1,71,APAC,3,3,1,notifications,6,chat,3,devops,2,1,3.328402,0,0,0,0,1,positive,3,306,low,1
3,1000000003,Wed,3,100003,Small,1,media,7,Plus,2,100,AMER,1,2,2,analytics,5,chat,3,finance,4,25,2.931906,16,0,0,0,1,neutral,2,363,medium,2
4,1000000004,Mon,1,100019,Small,1,ecommerce,2,Plus,2,332,AMER,1,5,1,analytics,5,web,1,support,1,28,3.255222,6,0,0,0,0,neutral,2,442,low,1


# **Escalamiento de variables númericas**

In [ ]:
#19 Aplicamos log1p(x) a 'customers_affected', error_rate_pct, and 'downtime_min'
dc['customers_affected_log'] = np.log1p(dc['customers_affected'])
dc['downtime_min_log'] = np.log1p(dc['downtime_min'])
dc['error_rate_pct_log'] = np.log1p(dc['error_rate_pct'])

print("Log1p transformación applicada a 'customers_affected', 'downtime_min' y error_rate_pct.")
display(dc[['customers_affected', 'customers_affected_log', 'downtime_min', 'downtime_min_log',"error_rate_pct","error_rate_pct_log"]].head())
print("Valores máximos transformados")
display(dc[['customers_affected', 'customers_affected_log', 'downtime_min', 'downtime_min_log',"error_rate_pct","error_rate_pct_log"]].max())

Log1p transformación applicada a 'customers_affected', 'downtime_min' y error_rate_pct.


,customers_affected,customers_affected_log,downtime_min,downtime_min_log,error_rate_pct,error_rate_pct_log
0,2,1.098612,6,1.945910,5.451201,1.864266
1,24,3.218876,2,1.098612,2.495538,1.251487
2,1,0.693147,0,0.000000,3.328402,1.465198
3,25,3.258097,16,2.833213,2.931906,1.369124
4,28,3.367296,6,1.945910,3.255222,1.448147


Valores máximos transformados


,0
customers_affected,5757.000000
customers_affected_log,8.658345
downtime_min,196.000000
downtime_min_log,5.283204
error_rate_pct,79.814565
error_rate_pct_log,4.392157


In [ ]:
#20 aplicmo z-score a variabls numericas
from sklearn.preprocessing import StandardScaler

cols_to_scale = [
    "past_30d_tickets",
    "past_90d_incidents",
    "description_length",
    "downtime_min_log",
    "customers_affected_log",
    "error_rate_pct_log"
]

# Iniciamos StandardScaler
scaler = StandardScaler()

 #Aplicamos la escalación estsndard (z-score)
dc[cols_to_scale] = scaler.fit_transform(dc[cols_to_scale])

print("Datos escalados (primeras 5 filas):")
display(dc[cols_to_scale].head())

print("\nValores Min and Max después de escalar:")
display(dc[cols_to_scale].agg(['min', 'max']))

Datos escalados (primeras 5 filas):


,past_30d_tickets,past_90d_incidents,description_length,downtime_min_log,customers_affected_log,error_rate_pct_log
0,-0.652358,-1.257509,-1.368881,0.345205,-1.387750,0.553666
1,-0.652358,0.786423,0.874317,-0.206401,-0.228376,-0.302346
2,-0.293185,-0.576198,-0.611562,-0.921618,-1.609461,-0.003806
3,-0.652358,0.105113,-0.065142,0.922856,-0.206930,-0.138015
4,0.425160,-0.576198,0.692177,0.345205,-0.147219,-0.027626



Valores Min and Max después de escalar:


,past_30d_tickets,past_90d_incidents,description_length,downtime_min_log,customers_affected_log,error_rate_pct_log
min,-1.370703,-1.257509,-3.353248,-0.921618,-1.988478,-2.050592
max,5.453574,6.918218,4.267873,2.517845,2.745960,4.084965


# **Analisis de variabes para eliminar: Variables categóricas**

**company_id y org_users**
De acuerdo a las tablas company_id y org_users tienen una relación de 1:1, y el tamaño de la organización ya está reflejado en company_size, el número de tickets realizados no necesariamente reflejan la importancia que se les dara a los tickets como si puede hacer customer_tier. Las variables de Id pueden generar un over fiting

**Conclusión:** eliminar company_id y org_users  

**Conclusión:** Eliminamos region

# Manejo de ariables categóricas, (tidydata)

In [ ]:
# 28Apply one-hot encoding to categorical variables
cols_to_encode = ['industry', 'company_size', 'customer_tier', 'product_area']

# Use pd.get_dummies to apply one-hot encoding
dc = pd.get_dummies(dc, columns=cols_to_encode, drop_first=True, dtype=int)

print("DataFrame after one-hot encoding:")
dc.info()
display(dc.head())

DataFrame after one-hot encoding:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   past_30d_tickets            50000 non-null  float64 
 1   past_90d_incidents          50000 non-null  float64 
 2   booking_channel             50000 non-null  object  
 3   payment_impact_flag         50000 non-null  int64   
 4   security_incident_flag      50000 non-null  int64   
 5   data_loss_flag              50000 non-null  int64   
 6   has_runbook                 50000 non-null  int64   
 7   description_length          50000 non-null  float64 
 8   priority                    50000 non-null  category
 9   customers_affected_log      50000 non-null  float64 
 10  downtime_min_log            50000 non-null  float64 
 11  error_rate_pct_log          50000 non-null  float64 
 12  industry_fintech            50000 non-nu

,past_30d_tickets,past_90d_incidents,booking_channel,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,description_length,priority,customers_affected_log,downtime_min_log,error_rate_pct_log,industry_fintech,industry_gaming,industry_healthcare,industry_logistics,industry_media,industry_saas_b2b,company_size_Medium,company_size_Small,customer_tier_Enterprise,customer_tier_Plus,product_area_auth,product_area_billing,product_area_data_pipeline,product_area_mobile,product_area_notifications
0,-0.652358,-1.257509,web,0,0,0,0,-1.368881,low,-1.387750,0.345205,0.553666,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
1,-0.652358,0.786423,chat,0,0,0,0,0.874317,low,-0.228376,-0.206401,-0.302346,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
2,-0.293185,-0.576198,chat,0,0,0,1,-0.611562,low,-1.609461,-0.921618,-0.003806,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1
3,-0.652358,0.105113,chat,0,0,0,1,-0.065142,medium,-0.206930,0.922856,-0.138015,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0
4,0.425160,-0.576198,web,0,0,0,0,0.692177,low,-0.147219,0.345205,-0.027626,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


# Eliminar variables sobrantes

In [ ]:
# Eliminar la columna 'booking_channel'
delete=["ticket_id",
        "company_size_cat",
        "industry_cat",
        "customer_tier_cat",
        "region_cat",
        "product_area_cat",
        "booking_channel_cat",
        "reported_by_role_cat",
        "customer_sentiment_cat",
        "priority_cat",
        "day_of_week_num",
        "customers_affected",
        "downtime_min",
        "error_rate_pct",
        'company_id',
        'org_users',
        'region',
        'day_of_week',
        'customer_sentiment',
        'reported_by_role',
        'booking_channel']


dc.drop(delete, inplace=True)

print("Columnas restantes en el DataFrame dc después de eliminar variables:")
print(dc.columns.tolist())

print("\nNúmero de subcategorías por columna categórica:")

# Identificar columnas categóricas restantes
categorical_cols_remaining = dc.select_dtypes(include=['object', 'category']).columns

for col in categorical_cols_remaining:
    print(f"\n--- Columna: {col} ---")
    print(f"Número de subcategorías: {dc[col].nunique()}")
    print("Subcategorías:")
    display(dc[col].value_counts().sort_index())

Columnas restantes en el DataFrame dc después de eliminar booking_channel:
['past_30d_tickets', 'past_90d_incidents', 'payment_impact_flag', 'security_incident_flag', 'data_loss_flag', 'has_runbook', 'description_length', 'priority', 'customers_affected_log', 'downtime_min_log', 'error_rate_pct_log', 'industry_fintech', 'industry_gaming', 'industry_healthcare', 'industry_logistics', 'industry_media', 'industry_saas_b2b', 'company_size_Medium', 'company_size_Small', 'customer_tier_Enterprise', 'customer_tier_Plus', 'product_area_auth', 'product_area_billing', 'product_area_data_pipeline', 'product_area_mobile', 'product_area_notifications']


# **Balanceo de clase Target: priority**

In [ ]:
#29 verificamos frecuencias de priority
print("Distribución de la columna 'priority':")
display(dc['priority'].value_counts())
print("\nDistribución de la columna 'priority' (%):")
display(dc['priority'].value_counts(normalize=True) * 100)

Distribución de la columna 'priority':


,count
priority,
low,25000
medium,17500
high,7500



Distribución de la columna 'priority' (%):


,proportion
priority,
low,50.0
medium,35.0
high,15.0


**Conclusión:** Aplicar Classs-Weight para ecitar que el modelo le de preferencia a low, cuando los importantes son high

In [ ]:

#30 balanceamos priority con class_weight
from sklearn.utils import class_weight
import numpy as np

# Calculate class weights
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(dc['priority'].astype(str)), # Convert unique classes to numpy array of strings
    y=dc['priority'].astype(str) # Also convert y to strings to match classes
)

# Create a dictionary of class weights
class_weight_dict = {
    cls: weight for cls, weight in zip(np.unique(dc['priority'].astype(str)), class_weights) # Use unique classes as strings for dictionary keys
}

print("Class weights for 'priority':")
display(class_weight_dict)

Class weights for 'priority':


{'high': np.float64(2.2222222222222223),
 'low': np.float64(0.6666666666666666),
 'medium': np.float64(0.9523809523809523)}